In [424]:
import csv
import numpy as np
from sklearn import metrics
import math
import operator
from scipy.spatial import distance
import time

In [425]:
def get_id():
    print('Name: Hieu Khuong, CIS4526')
    return 'tug60121'

In [426]:
#KNN ALGORITHM

In [427]:
def compute_accuracy(test_y, pred_y):
    accuracy = metrics.accuracy_score(test_y, pred_y)
    return accuracy

In [428]:
#Get the majority value of neighbors array
def getResponse(neighbors):
	classVotes = {}
	for x in range(len(neighbors)):
		response = neighbors[x]
		if response in classVotes:
			classVotes[response] += 1
		else:
			classVotes[response] = 1
	sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
	return sortedVotes[0][0]

In [429]:
#Get an array of the neighbor values
def getNeighbors(trainingSet, testInstance, k, trainY):
    distances = []
    for x in range(len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x])
        distances.append((trainY[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

In [430]:
def euclideanDistance(instance1, instance2):
    dst = distance.euclidean(instance1, instance2)
    return dst

In [431]:
def test_knn(trainX,trainY, test_x, num_nn):
    pred_y = np.zeros(len(test_x))
    for x in range(len(test_x)):
        neighbors = getNeighbors(trainX, test_x[x], num_nn, trainY)
        result = getResponse(neighbors)
        pred_y[x] = result
    return pred_y

In [432]:
def confusionMatrix(testY, pred_y):
    return metrics.confusion_matrix(testY,pred_y)

In [433]:
#END OF KNN ALGORITHM

In [434]:
#POCKET ALGORITHM

In [435]:
def convert_label(label, train_y):
    train = np.zeros(len(train_y))
    for i in range(len(train_y)):
        if train_y[i] == label:
            train[i] = 1
        else:
            train[i] = -1
    return train

In [436]:
def sign_function(weight, train_point):
    if np.dot(weight[1:], train_point) + weight[0] < 0:
        return -1
    else:
        return 1

In [437]:
def train_pocket(train_x, train_y, num_iters):
    # Initialize w
    # wTx is the (signed) distance of point x to hyperplane w
    num_train_data = len(train_x[0])  # grab number of columns
    weight = np.random.rand(num_train_data + 1)
    bestWeight = weight
    currentBestAccuracy = compute_accuracy(train_y,test_pocket(weight,train_x))
    for step in range(num_iters): 
        for j in range(len(train_x)):
            y_n = sign_function(weight,train_x[j])
            if (y_n != train_y[j]):
                weight[1:] = weight[1:] + train_y[j] * train_x[j]

        updatedAccuracy = compute_accuracy(train_y,test_pocket(weight,train_x))
        if(updatedAccuracy>currentBestAccuracy):
            bestWeight = weight
            currentBestAccuracy = updatedAccuracy
    
    return bestWeight

In [438]:
def test_pocket(w, test_x):
    pred_y = np.zeros(len(test_x))
    for i in range(len(test_x)):
         pred_y[i] = sign_function(w,test_x[i])
    return pred_y

In [439]:
#Return predicted y vector after using a matrix of best weights from Pocket
def OVA(weight,testX):
    pred_y = np.zeros(len(testX))
    count=0
    for x_vector in testX:
        bestGuess = 0
        maxDistance = np.dot(weight[0][1:], x_vector) + weight[0][0]
        for i in range(len(weight)):
            if((np.dot(weight[i][1:], x_vector) + weight[i][0]) > maxDistance):
                maxDistance = np.dot(weight[i][1:], x_vector) + weight[i][0]
                bestGuess = i
        pred_y[count] = bestGuess
        count+=1
    return pred_y

In [440]:
def main():
    
    #PROFESSOR PROVIDED PART
    
    # Read the data file
    szDatasetPath = './letter-recognition.data' # Put this file in the same place as this script
    listClasses = []
    listAttrs = []
    with open(szDatasetPath) as csvFile:
        csvReader = csv.reader(csvFile, delimiter=',')
        for row in csvReader:
            listClasses.append(row[0])
            listAttrs.append(list(map(float, row[1:])))
    # Generate the mapping from class name to integer IDs
    mapCls2Int = dict([(y, x) for x, y in enumerate(sorted(set(listClasses)))])
    # Store the dataset with numpy array
    dataX = np.array(listAttrs)
    dataY = np.array([mapCls2Int[cls] for cls in listClasses])
    # Split the dataset as the training set and test set
    nNumTrainingExamples = 15000
    trainX = dataX[:nNumTrainingExamples, :]
    trainY = dataY[:nNumTrainingExamples]
    testX = dataX[nNumTrainingExamples:, :]
    testY = dataY[nNumTrainingExamples:]

    num_train = nNumTrainingExamples
    num_test = len(dataX)-nNumTrainingExamples
    num_dims = trainY.shape[0]
    
    #MY PART
    k_range = {1,3,5,7,9}
    subsampleSize = [100,1000,2000,5000,10000,15000]
    
    #DO 30 RUNS OF KNN WITH DIFFERENT SUBSAMPLE SIZE
    print("THIS IS KNN ALGORITHM")
    for subSize in subsampleSize:
        #Declate subsample for training
        tempTrainX = trainX[:subSize,:]
        tempTrainY = trainY[:subSize]
        start = time.time()
        #Try all the numbers of neighbor 
        for k in k_range:
            startTime = time.time()
            pred_y = test_knn(tempTrainX,tempTrainY,testX,k)
            print('KNN:','Subsample train size', subSize, ', amount of neighbor',k)
            print('Accuracy:',compute_accuracy(testY,pred_y))
            print('Time taken:', time.time()-startTime)

    
    #RUN POCKET ALGORITHM FOR MULTIPLE SUBSAMPLE SIZE, THEN RUN OVA WITH THE OBTAINED WEIGHTS
    print("THIS IS OVA USING POCKET ALGORITHM")
    
    subsampleSize2 = [100,1000,2000,5000,10000,15000]
    for subSize in subsampleSize2:
        tempTrainX = trainX[:subSize,:]
        tempTrainY = trainY[:subSize]   
        
        #List to store weights for each Perceptron training on a letter number
        mylist = []
        
        startTime = time.time()
        #Find the best weights for each letters using Pocket Algorithm
        for i in range(0,26):
            
            #Convert label to each letter 0-25
            tempTrainYConvert = convert_label(i,tempTrainY)
            testYConvert = convert_label(i,testY)
            
            #Train to learn best weight using Pocket
            w = train_pocket(tempTrainX,tempTrainYConvert,10)
            mylist.append(w)    
        
        #Use matrix of best weights from above to predict multiclass
        weights = np.array(mylist) 
        ova_pred_y = OVA(weights,testX)
        print('OVA:','Subsample train size', subSize)
        print('Accuracy of OVA:',compute_accuracy(testY,ova_pred_y))
        print('Time taken:', time.time()-startTime)
    
    #Make a confusion matrix
    trainXconf = dataX[:2000,:]
    trainYconf = dataY[:2000]
    conf_pred_y = test_knn(trainXconf,trainYconf,testX,3)
    f = open('confu-matrix.txt','w+')
    confusionmatrix = confusionMatrix(testY,conf_pred_y)
    for i in confusionmatrix:
        f.write(str(i))
    f.close()
    
    return None

if __name__ == "__main__":
    main()

THIS IS KNN ALGORITHM
KNN: Subsample train size 100 , amount of neighbor 1
Accuracy: 0.3456
Time taken: 7.366750717163086
KNN: Subsample train size 100 , amount of neighbor 3
Accuracy: 0.3378
Time taken: 6.717294931411743
KNN: Subsample train size 100 , amount of neighbor 5
Accuracy: 0.3122
Time taken: 6.871142148971558
KNN: Subsample train size 100 , amount of neighbor 7
Accuracy: 0.2882
Time taken: 6.745088577270508
KNN: Subsample train size 100 , amount of neighbor 9
Accuracy: 0.264
Time taken: 6.999065160751343
KNN: Subsample train size 1000 , amount of neighbor 1
Accuracy: 0.7568
Time taken: 67.69041085243225
KNN: Subsample train size 1000 , amount of neighbor 3
Accuracy: 0.7322
Time taken: 65.57635116577148
KNN: Subsample train size 1000 , amount of neighbor 5
Accuracy: 0.7144
Time taken: 79.12040877342224
KNN: Subsample train size 1000 , amount of neighbor 7
Accuracy: 0.6886
Time taken: 74.68391728401184
KNN: Subsample train size 1000 , amount of neighbor 9
Accuracy: 0.669
Time 